<h3>Лабораторная работа №5</h3>
<p style="font-size: 16px">Исследование вероятности экспозиции совместно с вероятностью блокировки
на основе модели антенн</p>
<hr>
<p style="font-size: 14px">Цель:</p>
<ol>
    <li>Исследование вероятности экспозиции на основе модели антенн.</li>
</ol>
<hr>
<p><b><u>Задание</u></b></p>
<p style="width: 60%">В условиях предыдущей лабораторной работы №4, на двух точках, между
которыми производится проверка прямой видимости, формируется диаграмма
направленности, задающаяся вектором направления и углом. Предполагается, что
если для обеих из рассматриваемых точек вторая точка лежит внутри сектора,
описанного вектором и углом и линия прямой видимости между ними не
заблокирована, то точки находятся в области видимости друг друга.</p>
<p style="width: 60%">При условии, что направление вектора диаграммы задается случайно
равномерно распределенной величиной от 0 до 2 Pi, а угол является константой,
определить находятся ли рассматриваемые точки в области видимости друг друга.</p>
<p style="width: 60%">На основе 1000 реализаций определить вероятность нахождения точек в
области видимости друг друга. Полученный результат сравнить с результатами
математической модели.</p>
<hr>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math as m

In [4]:
# Вводим константы (честно украдены с 4ой лабораторной)
A = 10   # Сторона квадрата
q = 0.2  # Плотность блокирующих объектов
r = 0.2  # Радиус окружности блокирующего объекта
x = 4    # Расстояние между двумя точками для 1ого задания

# Далее добавим константный угол в радианах
VAR_ANGLE_RAD = 0.7

# Далее не менее честно крадём функцию получения блокирующих объектов
def giveMeLockingObjects():
    xDots = np.random.uniform(low = 0, high = A, size = int(q * A ** 2))
    yDots = np.random.uniform(low = 0, high = A, size = int(q * A ** 2))
    return xDots, yDots

# Не забываем прихватить с собой функцию на проверку блокировки между двумя точками
def checkIfLocked(xFirst, yFirst, xSecond, ySecond, xLockDots, yLockDots):
    xToCheck = np.linspace(xFirst, xSecond, 100)
    yToCheck = np.linspace(yFirst, ySecond, 100)

    for j in range(len(xToCheck)):
        for i in range(len(xLockDots)):
            if (m.sqrt((xToCheck[j] - xLockDots[i]) ** 2 + (yToCheck[j] - yLockDots[i]) ** 2) < r): return False

    return True

def checkIfVisible(xFirst, yFirst, xSecond, ySecond):
    angleFirstToSecond = m.atan2(yFirst - ySecond, xFirst - xSecond)

    # Для диаграммы направленности будем генерировать ее "потолок", а "пол" будем получать вычитанием константного угла
    angleGeneratedFirst = np.random.uniform(low = - m.pi, high = m.pi)
    angleGeneratedSecond = angleGeneratedFirst - VAR_ANGLE_RAD

    if ( (angleGeneratedFirst > angleFirstToSecond) and (angleGeneratedSecond < angleFirstToSecond) ):
        return True
    
    if (angleGeneratedSecond < - m.pi) \
        and ( (angleFirstToSecond < angleGeneratedFirst) or (angleFirstToSecond > angleGeneratedSecond + 2 * m.pi) ):
        return True

    return False

# Модифицируем проверку и расчёт вероятности из прошлой лабораторной так,
# чтобы проверка и расчёт учитывали диаграмму направленности
totalChecks = 10000
totalLocks  = 0

mathP = 0

xLocks, yLocks = giveMeLockingObjects()

for i in range(totalChecks):
    # Генерируем точки
    xFirst = np.random.uniform(low = 0, high = A)
    yFirst = np.random.uniform(low = 0, high = A)
    xSecond = np.random.uniform(low = 0, high = A)
    ySecond = np.random.uniform(low = 0, high = A)
    
    if (not checkIfLocked(xFirst, yFirst, xSecond, ySecond, xLocks, yLocks)
       or not checkIfVisible(xFirst, yFirst, xSecond, ySecond)
       or not checkIfVisible(xSecond, ySecond, xFirst, yFirst)):
        totalLocks += 1

    # Расчёт вероятности по формуле, для согласования со случайным расстоянием между точками
    mathP += ((1 - m.exp(- q * 2 * r * ( m.sqrt( (xFirst - xSecond) ** 2 + (yFirst - ySecond) ** 2 ) + r ) )) / totalChecks) \
        + (m.exp(- q * 2 * r * ( m.sqrt( (xFirst - xSecond) ** 2 + (yFirst - ySecond) ** 2 ) + r ) ) / totalChecks) \
        * (1 - (VAR_ANGLE_RAD / 2 / m.pi) ** 2)

print('Экспериментальный результат вероятности блокировки = %.3f' %(totalLocks / totalChecks))
print('Теоретический результат блокировки %.3f' %(mathP))

Экспериментальный результат вероятности блокировки = 0.992
Теоретический результат блокировки 0.992


<hr>
<h3>Выводы:</h3>
<ol>
    <li>Исследование вероятности экспозиции на основе модели антенн.</li>
    <li>Сравнили генеративные и теоретические вероятности эскпозиций</li>
</ol>